### 1. Откроем файл с данными и изучим общую информацию. 

In [2]:
import pandas as pd
customers_data = pd.read_csv('/datasets/data.csv')

In [3]:
customers_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [4]:
customers_data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


### Вывод

У стажа были замечены пропуски, отрицательные и аномально высокие значения. Т.к. стаж никак не связан с конечной целью исследования, то легче будет просто удалить данный столбец. Также замечены пропуски в столбце с заработком. Замечены разные регистры в столбце 'образование' и не категоризированные данные у целей кредита.

### 2. Предобработка данных

### Обработка пропусков

Посмотрим на количество пропусков в таблице и где они находятся с помощью метода isnull() и sum()

In [5]:
customers_data.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Для столбца 'total_income' заполним пропуски медианной зарплатой данного столбца т.к. для зарплаты лучше всего считать медиану, для этого подсчитаем её методом median.()

In [6]:
income_median = customers_data['total_income'].median()

In [7]:
income_median

145017.93753253992

Теперь методом fillna() заполним пропуски в столбце total_income нашей медианой

In [8]:
customers_data['total_income'] = customers_data['total_income'].fillna(income_median)

In [10]:
customers_data.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income           0
purpose                0
dtype: int64

Так как мы решили, что столбец, отражающий стаж нам не нужен, можно его удалить методом drop()

In [11]:
customers_data.drop(['days_employed'], axis = 'columns', inplace = True)

In [12]:
customers_data.isnull().sum()

children            0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Вывод

Мы удалили ненужный столбец и заменили все пропуски в столбце 'заработок' на их медиану, что позволяет не переживать по поводу пропусков и работать дальше. Причины возникновения пропусков возможно связаны с технической частью. Возможно часть пропусков в стаже повредила данные в столбце с заработком, т.к. пропуски находятся на одних и тех же строках.

### Замена типа данных

Проверим типы данных у все таблицы, нет ли в них странностей

In [13]:
customers_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 11 columns):
children            21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(1), int64(5), object(5)
memory usage: 1.8+ MB


Ничего особенного. Но для удобства, тип данных лучше заменить только у одного столбца total_income. Применим для этого метод astype() с аргументом int

In [14]:
customers_data['total_income'] = customers_data['total_income'].astype('int')

In [15]:
customers_data['total_income'].dtype

dtype('int64')

In [16]:
customers_data['total_income'].head()

0    253875
1    112080
2    145885
3    267628
4    158616
Name: total_income, dtype: int64

### Вывод

Теперь заработок является целочисленным значением и стал удобен для восприятия.

### Обработка дубликатов

В начале говорилось о том, что в столбце 'образование' разные регистры, посмотрим на них полностью

In [17]:
customers_data['education'].value_counts()

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64

Здесь достаточно снизить всем регистр с помощью методы str.lower() чтобы разобраться с дубликатами

In [18]:
customers_data['education'] = customers_data['education'].str.lower()

In [19]:
customers_data['education'].value_counts()

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

### Вывод

Теперь столбец с образованием после обработки дубликатов стал удобнее и с ним можно проводить точные математические действия. Дубликаты наверняка связаны с человеческим фактором, сотрудники вносили данные в базу данных по разному.

### Лемматизация

Посмотрим на все существующие значения в столбце с целями кредита

In [20]:
customers_data['purpose'].value_counts()

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
операции с жильем                         653
покупка жилья для сдачи                   653
операции с коммерческой недвижимостью     651
покупка жилья                             647
жилье                                     647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
строительство недвижимости                620
покупка своего жилья                      620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

Проведём лемматизацию столбца, для определения вывода имён существительных. Вызовем библиотеку

In [21]:
from pymystem3 import Mystem
m = Mystem()

Создадим функцию, в которой каждое значение в столбце будет лемматизировано с помощью цикла и напечатано в отдельном новом столбце с помощью метода apply()

In [22]:
def purpose_group(row):
    for query in row.split():
        lemmas = ' '.join(m.lemmatize(row))
    return lemmas  

In [23]:
customers_data['new_purpose'] = customers_data['purpose'].apply(purpose_group)

In [24]:
customers_data['new_purpose'].value_counts()

автомобиль \n                                    973
свадьба \n                                       797
на   проведение   свадьба \n                     777
сыграть   свадьба \n                             774
операция   с   недвижимость \n                   676
покупка   коммерческий   недвижимость \n         664
покупка   жилье   для   сдача \n                 653
операция   с   жилье \n                          653
операция   с   коммерческий   недвижимость \n    651
жилье \n                                         647
покупка   жилье \n                               647
покупка   жилье   для   семья \n                 641
строительство   собственный   недвижимость \n    635
недвижимость \n                                  634
операция   со   свой   недвижимость \n           630
строительство   жилой   недвижимость \n          626
покупка   недвижимость \n                        624
покупка   свой   жилье \n                        620
строительство   недвижимость \n               

### Вывод

Из данных легко заметить, что клиенты из данной выборки берут кредиты на автомобиль, образование, недвижимость и свадьбу. Благодаря лемматизации, теперь можно категоризировать цели кредита

### Категоризация данных

Категоризируем данные в столбце цели кредита, для определения категории целей. Это понадобится для ответа на один из вопросов. Мы уже узнали, что данные делятся на четыре категории

Напишем функцию, в которой с помощью цикла категоризируем цели кредита с помощью нескольких условий if, in и запишем их в новый столбец

In [25]:
def group(row):
    for word in row:
        if 'автомобиль' in row:
            return 'автомобиль'
        if 'свадьба' in row:
            return 'свадьба'
        if 'жилье' in row:
            return 'недвижимость'
        if 'недвижимость' in row:
            return 'недвижимость'
        if 'образование' in row:
            return 'образование'

In [26]:
customers_data['final_purpose'] = customers_data['new_purpose'].apply(group)

Выполним проверку с помощью метода value_counts()

In [27]:
customers_data['final_purpose'].value_counts()

недвижимость    10840
автомобиль       4315
образование      4022
свадьба          2348
Name: final_purpose, dtype: int64

Удалим наш лемматизированный столбец

In [28]:
customers_data.drop(['new_purpose'], axis = 'columns', inplace = True)

Цели кредиты категоризированы, теперь необходимо сделать тоже самое с заработком

Категоризируем данные в столбце цели заработок, для определения категорий заработка. Это понадобится для ответа на один из вопросов. Необходимо сначала узнать, на сколько категорий разделить данные. Первым делом посмотрим, сколько у нас всего зарплат. Глянем, сколько всего клиентов находится в таблице

In [29]:
print(len(customers_data))

21525


Теперь посмотрим, сколько людей зарабатывают меньше 100000

In [30]:
print(customers_data[customers_data['total_income'] <=100000]['total_income'].count())

4463


Из общего количества довольно мало, теперь глянем, сколько клиентов зарабатывает больше 200000

In [31]:
print(customers_data[customers_data['total_income'] >=200000 ]['total_income'].count())

5066


Тоже немного, решаем сделать три категории заработка: низкий, средний и высокий.

In [32]:
def income_group(total_income):
    if total_income < 100000:
        return 'низкий'
    if total_income < 200000:
        return 'средний'
    if total_income >= 200000:
        return 'высокий'

customers_data['text_income'] = customers_data['total_income'].apply(income_group)

In [33]:
print(customers_data['text_income'].value_counts())

средний    11996
высокий     5066
низкий      4463
Name: text_income, dtype: int64


### Вывод

Мы выяснили, что если сложить цели кредита, такие как свадьба, автомобиль и образование, то даже их сумма будет меньше одной недвижимости. Кредиты на недвижимость довольно популярны в данном банке. В заработке точно также сильно доминирует средний доход. Категоризация целей кредита и заработка поможет ответить нам на часть запросов от руководства

### 3. Проведём исследовательский анализ данных

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

Посмотрим, на таблицу с детьми

In [34]:
print(customers_data['children'].value_counts())

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64


В данных обнаружено два артефакта: -1 и 20 детей. Возможно, часть единичных значений стала отрицательной из-за выгрузки, поэтому заменим их на положительное число. Применим для этого метод replace()

In [35]:
customers_data['children'] = customers_data['children'].replace(-1, 1)

In [36]:
print(customers_data['children'].value_counts())

0     14149
1      4865
2      2055
3       330
20       76
4        41
5         9
Name: children, dtype: int64


От -1 избавились, а как быть с 20? Их не так много, поэтому, их можно удалить. На конечный результат для наших целей это сильно не скажется. Применим для этого фильтр

In [37]:
customers_data = customers_data[customers_data['children'] != 20]

In [38]:
print(customers_data['children'].value_counts())

0    14149
1     4865
2     2055
3      330
4       41
5        9
Name: children, dtype: int64


После этих действий, приступим к изучению вопроса.

In [39]:
customers_data.groupby('debt')['children'].value_counts()

debt  children
0     0           13086
      1            4420
      2            1861
      3             303
      4              37
      5               9
1     0            1063
      1             445
      2             194
      3              27
      4               4
Name: children, dtype: int64

Сделаем из них сводную таблицу с помощью pivot_table(). В качестве основы возьмём количество детей, колонками сделаем значения долга, в качестве значения сделаем цели кредита, а к функции применим count т.е. подсчёт значений

In [40]:
children_debt = customers_data.pivot_table(index = 'children', columns = 'debt', values = 'purpose', aggfunc = 'count')

In [41]:
children_debt

debt,0,1
children,,
0,13086.0,1063.0
1,4420.0,445.0
2,1861.0,194.0
3,303.0,27.0
4,37.0,4.0
5,9.0,NaN


Исправим столбцы с помощью set_axis

In [42]:
children_debt.set_axis(['no_debt', 'debt'], axis = 'columns', inplace = True)

Посчитаем процент невозврата кредита по каждой группе

In [43]:
children_debt['%'] = children_debt['debt'] / (children_debt['debt'] + children_debt['no_debt']) * 100

Выведем финальную таблицу, с сортировкой по процентам, применив метод sort_values()

In [44]:
children_debt.sort_values(by = '%', ascending = False)

,no_debt,debt,%
children,,,
4,37.0,4.0,9.756098
2,1861.0,194.0,9.440389
1,4420.0,445.0,9.146968
3,303.0,27.0,8.181818
0,13086.0,1063.0,7.512898
5,9.0,NaN,NaN


### Вывод

Из таблицы видно, что кредиты чаще всего возвращают клиенты, имеющие пятерых детей, а чаще всего не отдают кредит в срок клиенты имеющие четырех детей. Если не брать в расчёт клиентов с пятью детьми, коих довольно мало, то чаще всего кредит возвращают люди не имеющие детей, а клиенты с тремя детьми возвращают кредит чаще, чем клиенты с одним, двумя или четырьмя. В целом можно сказать, что между наличием детей и возвращенем кредита присутствует небольшая зависимость. Клиентов без детей гораздо больше и они возвращают кредиты в срок чаще.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

Посмотрим на данные, разбитые по группам, применив метод groupby()

In [45]:
customers_data.groupby('family_status')['debt'].value_counts()

family_status          debt
Не женат / не замужем  0        2531
                       1         273
в разводе              0        1109
                       1          84
вдовец / вдова         0         893
                       1          63
гражданский брак       0        3780
                       1         385
женат / замужем        0       11403
                       1         928
Name: debt, dtype: int64

Сделаем из них сводную таблицу с помощбю pivot_table(). В качестве основы возьмём семейное положение, колонками сделаем значения долга, в качестве значения сделаем цели кредита, а к функции применим count

In [46]:
family_debt = customers_data.pivot_table(index = 'family_status', columns = 'debt', values = 'purpose', aggfunc = 'count')

In [47]:
family_debt

debt,0,1
family_status,,
Не женат / не замужем,2531,273
в разводе,1109,84
вдовец / вдова,893,63
гражданский брак,3780,385
женат / замужем,11403,928


Исправим названия столбцов с помощью set_axis

In [48]:
family_debt.set_axis(['no_debt', 'debt'], axis = 'columns', inplace = True)

Теперь подсчитаем процент невозврата кредита по каждой группе

In [49]:
family_debt['%'] = family_debt['debt'] / (family_debt['debt'] + family_debt['no_debt']) * 100

Выведем финальную таблицу и отсортируем проценты с помощью метода sort_values()

In [50]:
family_debt.sort_values(by = '%', ascending = False)

,no_debt,debt,%
family_status,,,
Не женат / не замужем,2531,273,9.736091
гражданский брак,3780,385,9.243697
женат / замужем,11403,928,7.525748
в разводе,1109,84,7.041073
вдовец / вдова,893,63,6.589958


### Вывод

Судя по данным, кредит чаще всего возвращают клиенты в разводе или потерявшие вторую половину. Не возвращают кредиты в срок клиенты, находящиеся в гражданском браке или свободные. Кредиты лучше давать тем, кто состоит в стандартном браке, т.к. если сравнивать с общим количеством каждой группы, то кредиты они возвращают довольно часто. Клиенты в разводе чаще возвращают кредит, возможно из-за развода во время гашения кредита и они его делят. А те кто потерял вторую половину, скорее всего могут продать из вещей что то уже не нужное и тем самым расплатиться по долгам. 

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Сначала посмотрим на общие данные

In [51]:
customers_data.groupby('text_income')['debt'].value_counts()

text_income  debt
высокий      0        4690
             1         356
низкий       0        4098
             1         354
средний      0       10928
             1        1023
Name: debt, dtype: int64

Сделаем сводную таблицу для наглядности с помощью pivot_table(). В качестве основы возьмём заработок, колонками сделаем значения долга, в качестве значения сделаем цели кредита, а к функции применим count

In [52]:
income_debt = customers_data.pivot_table(index = 'text_income', columns = 'debt', values = 'purpose', aggfunc = 'count').astype('float')

In [53]:
income_debt

debt,0,1
text_income,,
высокий,4690.0,356.0
низкий,4098.0,354.0
средний,10928.0,1023.0


Заменим названия столбцов с помощью set_axis()

In [54]:
income_debt.set_axis(['no_debt', 'debt'], axis = 'columns', inplace = True)

Как и раньше, для каждой категории посчитаем процент возврата кредита

In [55]:
income_debt['%'] = income_debt['debt'] / (income_debt['debt'] + income_debt['no_debt']) * 100

Применим метод sort_values() на проценты

In [56]:
income_debt.sort_values(by = '%', ascending = False)

,no_debt,debt,%
text_income,,,
средний,10928.0,1023.0,8.559953
низкий,4098.0,354.0,7.951482
высокий,4690.0,356.0,7.055093


### Вывод

Разница в 1-2% между уровнем дохода и возвращением кредита в срок говорит о совсем небольшой зависимости. Клиенты с высоким уровнем дохода, как и ожидается, возвращают кредит чаще, чем клиенты со средним доходом. Но при этом люди со средним доходом не возвращают кредит в срок так часто, как это делают люди с низким доходом.

- Как разные цели кредита влияют на его возврат в срок?

In [57]:
customers_data.groupby('final_purpose')['debt'].value_counts()

final_purpose  debt
автомобиль     0        3898
               1         401
недвижимость   0       10024
               1         780
образование    0        3638
               1         369
свадьба        0        2156
               1         183
Name: debt, dtype: int64

Седлаем сводную таблицу на основе этих данных, заменим названия столбцов, посчитаем процент по невозврату кредита в срок и отсортируем по процентам

In [58]:
purpose_debt = customers_data.pivot_table(index = 'final_purpose', columns = 'debt', values = 'purpose', aggfunc = 'count')

In [59]:
purpose_debt

debt,0,1
final_purpose,,
автомобиль,3898,401
недвижимость,10024,780
образование,3638,369
свадьба,2156,183


In [60]:
purpose_debt.set_axis(['no_debt', 'debt'], axis = 'columns', inplace = True)

In [61]:
purpose_debt['%'] = purpose_debt['debt'] / (purpose_debt['debt'] + purpose_debt['no_debt']) * 100

In [62]:
purpose_debt.sort_values(by = '%', ascending = False)

,no_debt,debt,%
final_purpose,,,
автомобиль,3898,401,9.327751
образование,3638,369,9.208884
свадьба,2156,183,7.823856
недвижимость,10024,780,7.219548


### Вывод

Судя по данным, чаще всего кредиты задерживают на автомобили и образование. На недвижимость кредиты возвращают в срок чаще. С учётом большого количества кредитов на недвижимость, можно с уверенностью сказать, что недвижимость -  это самый выгодный вариант для банка.

### 4. Общий вывод

Идеальный клиент банка - человек в браке, без детей, с высоким доходом, который собирается взять кредит на покупку недвижимости. Такой клиент вернёт деньги банку с большей вероятностью.
У кого четыре или два ребёнка, низкий - средний доход, не находятся в браке и решили взять кредит на машину - такие клиенты вернут кредит в срок с довольно низкой вероятностью и лучше узнать о клиенте побольше информации.